In [1]:
import pandas as pd

In [2]:
meat = pd.read_csv("./0.csv", encoding="utf-8")
fish = pd.read_csv("./1.csv", encoding="utf-8")
agriculture = pd.read_csv("./2.csv", encoding="utf-8")
banchan = pd.read_csv("./3.csv", encoding="utf-8")
kimchi = pd.read_csv("./4.csv", encoding="utf-8")
snack = pd.read_csv("./5.csv", encoding="utf-8")
beverage = pd.read_csv("./6.csv", encoding="utf-8")
icecream = pd.read_csv("./7.csv", encoding="utf-8")
frozen = pd.read_csv("./8.csv", encoding="utf-8")
gagong = pd.read_csv("./9.csv", encoding="utf-8")
health = pd.read_csv("./10.csv", encoding="utf-8")
meat.head()

,Unnamed: 0,name
0,0,[맛있닭] 닭가슴살 볼 큐브 외 6종 1kg
1,1,[맛있닭] 닭가슴살스테이크 외 29종 골라담기
2,2,"바날라 보존료, 발색제 없는 촉촉한 닭가슴살(네 가지 맛 선택) 100g"
3,3,아임닭 닭가슴살 모음 골라담기
4,4,족만이 순살족발


In [3]:
print(len(meat["name"]), len(fish["name"]), len(agriculture["name"]), len(banchan["name"]), len(kimchi["name"]),
      len(snack["name"]), len(beverage["name"]), len(icecream["name"]), len(frozen["name"]), len(gagong["name"]), len(health["name"]))

4100 4095 4098 4100 4100 4100 4100 4100 4096 4098 4087


In [4]:
meat_df = pd.DataFrame(meat["name"])
meat_df["label"] = "meat"
fish_df = pd.DataFrame(fish["name"])
fish_df["label"] = "fish"
agriculture_df = pd.DataFrame(agriculture["name"])
agriculture_df["label"] = "agriculture"
banchan_df = pd.DataFrame(banchan["name"])
banchan_df["label"] = "banchan"
kimchi_df = pd.DataFrame(kimchi["name"])
kimchi_df["label"] = "kimchi"
snack_df = pd.DataFrame(snack["name"])
snack_df["label"] = "snack"
beverage_df = pd.DataFrame(beverage["name"])
beverage_df["label"] = "beverage"
icecream_df = pd.DataFrame(icecream["name"])
icecream_df["label"] = "icecream"
frozen_df = pd.DataFrame(frozen["name"])
frozen_df["label"] = "frozen"
gagong_df = pd.DataFrame(gagong["name"])
gagong_df["label"] = "gagong"
health_df = pd.DataFrame(health["name"])
health_df["label"] = "health"

In [6]:
data = meat_df.append(fish_df).append(agriculture_df).append(banchan_df).append(kimchi_df).append(snack_df).append(beverage_df).append(icecream_df).append(frozen_df).append(gagong_df).append(health_df)

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(data, test_size=0.33, random_state = 42)

In [9]:
train["label"] = pd.Categorical(train["label"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
train.groupby("label").count()

,name
label,
agriculture,2782
banchan,2734
beverage,2799
fish,2702
frozen,2769
gagong,2744
health,2772
icecream,2701
kimchi,2749


In [11]:
test.groupby("label").count()

,name
label,
agriculture,1316
banchan,1366
beverage,1301
fish,1393
frozen,1327
gagong,1354
health,1315
icecream,1399
kimchi,1351


In [12]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

In [14]:
cv = CountVectorizer(tokenizer=get_noun)

In [15]:
tdm = cv.fit_transform(train["name"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf_svm = Pipeline([('vect', CountVectorizer(tokenizer=get_noun)), ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(train["name"], train["label"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [17]:
predicted_svm = text_clf_svm.predict(test["name"])
np.mean(predicted_svm == test["label"])

0.841546218487395

In [18]:
text_clf_svm.predict(["묵은지"])

array(['kimchi'], dtype='<U11')

In [19]:
text_clf_svm.predict(["삽겹살"])

array(['meat'], dtype='<U11')

In [20]:
import pickle
from sklearn.externals import joblib 
# 객체를 pickled binary file 형태로 저장한다 
joblib.dump(text_clf_svm, "food_categorize_model.pkl") 

['food_categorize_model.pkl']